# New Southbound Eighteen Top-Ranked Products

In [1]:
import pandas as pd

path = 'C:/Users/2093/Desktop/Data Center/09. 產品別出口市場/comp_aggregate_6_utf-8.csv'
df = pd.read_csv(path, dtype={'product': str})

In [2]:
ctry_list = ['不丹', '印尼', '印度', '孟加拉', '寮國', '尼泊爾', '巴基斯坦', '斯里蘭卡', '新加坡',
             '柬埔寨', '汶萊', '泰國', '澳大利亞', '紐西蘭', '緬甸', '菲律賓', '越南', '馬來西亞']

# Filter for southbound eighteen and rank-1 products
rs = df.loc[(df['country'].apply(lambda x: x in ctry_list)) & (df['tw_rank'] == 1),
            ['country', 'product', 'desc2', 'desc4', 'desc6', 'val15', 'tw_val15', 'tw_share',
             'tw_rank']].sort_values(['country', 'tw_val15'], ascending=[True, False]).rename(
    columns={'country' : '進口國',
             'product' : '產品代碼',
             'desc2'   : 'HS2名稱',
             'desc4'   : 'HS4名稱',
             'desc6'   : 'HS6名稱',
             'val15'   : '自全球進口額(千美元)',
             'tw_val15': '自臺進口額(千美元)',
             'tw_share': '臺灣市占率',
             'tw_rank' : '臺灣排名'})

In [3]:
# Compute country summary
summary = rs.groupby('進口國').agg({'HS6名稱': 'count'}).rename(
    columns={'HS6名稱': '冠軍產品數'}).sort_values('冠軍產品數', ascending=False)

In [4]:
from pandas import ExcelWriter

# Write to separate worksheets
writer = ExcelWriter('eighteen.xlsx')
summary.to_excel(writer, sheet_name='總表')
for c in summary.index:
    rs[rs['進口國'] == c].to_excel(writer, sheet_name=c, index=False)
writer.save()

In [5]:
from openpyxl import load_workbook
from openpyxl.formatting.rule import DataBarRule

# Conditional formatting
wb = load_workbook('eighteen.xlsx')
rule_g = DataBarRule(start_type='percentile', start_value=10, end_type='percentile', end_value='90',
                     color='FF63C384', showValue='None', minLength=None, maxLength=None)
rule_b = DataBarRule(start_type='percentile', start_value=10, end_type='percentile', end_value='90',
                     color='FF638EC6', showValue='None', minLength=None, maxLength=None)
wb['總表'].conditional_formatting.add('B2:B16', rule_g)
for i, ctry in enumerate(wb.get_sheet_names()[1:]):
    n_product = summary['冠軍產品數'].values[i]
    wb[ctry].conditional_formatting.add('H2:H{}'.format(n_product + 1), rule_g)
    wb[ctry].conditional_formatting.add('G2:G{}'.format(n_product + 1), rule_b)
wb.save('eighteen.xlsx')